In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 64
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork_cos

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork_cos(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()[i]!=train_id)[0], replace=True)
        batch_features_negative[i] = x[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(0.9026, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5203166226912929
gzsl: unseen=0.0355 , seen=0.6761 , h=0.0675
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.8513, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5407211961301671
gzsl: unseen=0.0756 , seen=0.7180 , h=0.1369
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.8367, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.554089709762533
gzsl: unseen=0.0802 , seen=0.7342 , h=0.1446
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.8087, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5688654353562005
gzsl: unseen=0.0939 , seen=0.7412 , h=0

episode: 31000 loss tensor(0.8191, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5769569041336852
gzsl: unseen=0.3551 , seen=0.7650 , h=0.4851
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.7809, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.573966578715919
gzsl: unseen=0.3467 , seen=0.7638 , h=0.4769
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.7804, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5674582233948988
gzsl: unseen=0.3465 , seen=0.7616 , h=0.4763
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.7835, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5857519788918206
gzsl: unseen=0.3712 , seen=0.7620 , h=0.4992
__

episode: 61000 loss tensor(0.7756, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.533509234828496
gzsl: unseen=0.4090 , seen=0.7126 , h=0.5197
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.7752, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5255936675461741
gzsl: unseen=0.4077 , seen=0.7049 , h=0.5166
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.7784, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5266490765171504
gzsl: unseen=0.4077 , seen=0.7067 , h=0.5171
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.7798, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5386103781882146
gzsl: unseen=0.4118 , seen=0.7160 , h=0.5229
__

episode: 91000 loss tensor(0.7902, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5175021987686895
gzsl: unseen=0.4336 , seen=0.6791 , h=0.5293
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.7723, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5043095866314864
gzsl: unseen=0.4274 , seen=0.6636 , h=0.5200
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.7726, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5192612137203166
gzsl: unseen=0.4408 , seen=0.6727 , h=0.5326
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.8085, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5108179419525066
gzsl: unseen=0.4281 , seen=0.6757 , h=0.5242
_

episode: 121000 loss tensor(0.7735, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49498680738786277
gzsl: unseen=0.4241 , seen=0.6495 , h=0.5131
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.8064, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5099384344766931
gzsl: unseen=0.4391 , seen=0.6567 , h=0.5263
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.7711, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5014951627088831
gzsl: unseen=0.4332 , seen=0.6495 , h=0.5198
____________________________________________________________________________________________________
episode: 124000 loss tensor(0.7542, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5032541776605101
gzsl: unseen=0.4361 , seen=0.6468 , h=0.5

...

episode: 361000 loss tensor(0.7568, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5030782761653474
gzsl: unseen=0.4612 , seen=0.6107 , h=0.5255
____________________________________________________________________________________________________
episode: 362000 loss tensor(0.7615, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5014951627088831
gzsl: unseen=0.4600 , seen=0.6121 , h=0.5253
____________________________________________________________________________________________________
episode: 363000 loss tensor(0.7736, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5037818821459983
gzsl: unseen=0.4624 , seen=0.6103 , h=0.5262
____________________________________________________________________________________________________
episode: 364000 loss tensor(0.7643, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5004397537379067
gzsl: unseen=0.4600 , seen=0.6091 , h=0.52

episode: 391000 loss tensor(0.7663, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4956904133685136
gzsl: unseen=0.4549 , seen=0.6085 , h=0.5206
____________________________________________________________________________________________________
episode: 392000 loss tensor(0.7718, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49762532981530344
gzsl: unseen=0.4563 , seen=0.6081 , h=0.5214
____________________________________________________________________________________________________
episode: 393000 loss tensor(0.7769, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4990325417766051
gzsl: unseen=0.4573 , seen=0.6067 , h=0.5215
____________________________________________________________________________________________________
episode: 394000 loss tensor(0.7772, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5048372911169745
gzsl: unseen=0.4631 , seen=0.6113 , h=0.5

episode: 421000 loss tensor(0.7932, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49604221635883905
gzsl: unseen=0.4547 , seen=0.6061 , h=0.5196
____________________________________________________________________________________________________
episode: 422000 loss tensor(0.7657, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49551451187335094
gzsl: unseen=0.4547 , seen=0.6071 , h=0.5200
____________________________________________________________________________________________________
episode: 423000 loss tensor(0.7831, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4946350043975374
gzsl: unseen=0.4536 , seen=0.6077 , h=0.5195
____________________________________________________________________________________________________
episode: 424000 loss tensor(0.7907, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49551451187335094
gzsl: unseen=0.4545 , seen=0.6077 , h=0

episode: 451000 loss tensor(0.8226, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49393139841688655
gzsl: unseen=0.4533 , seen=0.6039 , h=0.5179
____________________________________________________________________________________________________
episode: 452000 loss tensor(0.7991, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49498680738786277
gzsl: unseen=0.4545 , seen=0.6031 , h=0.5184
____________________________________________________________________________________________________
episode: 453000 loss tensor(0.7781, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4909410729991205
gzsl: unseen=0.4517 , seen=0.6013 , h=0.5159
____________________________________________________________________________________________________
episode: 454000 loss tensor(0.8030, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4946350043975374
gzsl: unseen=0.4538 , seen=0.6037 , h=0.

episode: 481000 loss tensor(0.7904, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49797713280562883
gzsl: unseen=0.4588 , seen=0.6043 , h=0.5216
____________________________________________________________________________________________________
episode: 482000 loss tensor(0.7622, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.49973614775725594
gzsl: unseen=0.4605 , seen=0.6037 , h=0.5225
____________________________________________________________________________________________________
episode: 483000 loss tensor(0.7592, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5007915567282322
gzsl: unseen=0.4605 , seen=0.6071 , h=0.5237
____________________________________________________________________________________________________
episode: 484000 loss tensor(0.7779, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4993843447669305
gzsl: unseen=0.4593 , seen=0.6029 , h=0.